In [2]:
import pandas as pd, numpy as np, geopandas as gpd
import matplotlib.pyplot as plt
from importlib import reload
import functools
%matplotlib inline

pd.set_option('precision', 3)
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

In [19]:
import landmarks_integration_functions as lif
import utilities as uf
import landmarks_functions as lf

In [20]:
reload(lif)
reload(uf)

<module 'utilities' from 'C:\\Users\\g_filo01\\sciebo\\scripts\\Image of the City\\utilities.py'>

In [23]:
#initialise path, names, etc.

city_name = 'London'
epsg = 27700
crs = {'init': 'epsg:27700', 'no_defs': True}

directory_output ='Outputs/'+city_name+'/large_network/'+city_name
directory_intermediate = 'C:/Users/g_filo01/sciebo/Scripts/Image of the City/Outputs/'+city_name+'/intermediate/'+city_name

In [24]:
# try reading nodes, paths, landmarks and sight_lines
# At this point, every element has already been extracted

nodes = gpd.read_file(directory_output+'_nodes.shp', driver='ESRI Shapefile')
paths = gpd.read_file(directory_output+'_edges.shp', driver='ESRI Shapefile')
buildings = gpd.read_file(directory_output_inter+'_landmarks_noADV.shp', driver='ESRI Shapefile')

## Assigning buildings and scores to Nodes

In [25]:
# assign buildings to nodes
# buffer around the node that regulates the meaning of adjacent buildings
nodes = lif.assign_buildings_to_nodes(nodes, buildings, buffer = 80)

max_buffer = 2000 # max distance from node
threshold = 0.30 # min global score to define a global landmark
nodes = lif.assign_global_anchors(nodes, buildings, buffer = max_buffer, threshold = threshold)

In [ ]:
reload(lf)
# sight_lines = gpd.read_file(directory_intermediate+'_sightlines.shp')
# sight_lines_copy = lf.reassign_sight_lines_to_new_nodes(nodes, sight_lines)
# sight_lines_copy[['nodeID', 'buildingID', 'geometry']].to_file(directory_output+'_sight_lines.shp', driver='ESRI Shapefile')

In [40]:
sight_lines = gpd.read_file(directory_output+'_sight_lines.shp', driver='ESRI Shapefile')
nodes = lif.assign_3dvisible_landmarks(nodes, buildings, sight_lines, threshold = threshold)

## 2d advance Visibility Matrix

In [32]:
v = lif.visibility_2d(nodes, buildings, distance_along = 50, max_distance_node_to_building = 300)

done


In [34]:
matrix2d = lif.visibility_matrix2d(buildings, nodes, v)

In [37]:
matrix2d.to_csv(directory_output+'_visibility_matrix.csv')

## Conversion from lists to string, to export the file

In [43]:
to_convert = ['loc_land', 'loc_scor', 'dist_land', 'dist_scor', 'anchors', 'distances']
nodes_string = nodes.copy()
for column in to_convert: nodes_string[column] = nodes_string[column].astype(str)
nodes_string.to_file(directory_output+'_nodes.shp', driver='ESRI Shapefile')